<a href="https://colab.research.google.com/github/crystdang/Amazon-Vine-Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|5          |0            |0          |N   |Y                |
|RF0D1LEIF6L7  |4          |0            |0          |N   |Y                |
|RM6YKIWQVNSY  |1          |6            |6          |N   |Y                |
|R1RL3L68ASPS36|4          |0            |0          |N   |Y                |
|R1U4XFBFAG34CY|5          |0            |0          |N   |Y                |
|R3KFIK8P0I91PL|5          |0            |0          |N   |Y                |
|RENOAY76PPK1O |5          |0            |0          |N   |Y                |
|RINV884I0NL5V |1          |0            |0          |N   |Y                |
|R5KJH6CXZH2PX |5          |0            |0          |N   |Y                |
|RO69JF6QWD0W1 |5          |0            |0          |N   |Y    

In [6]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count
# is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
votes_df = vine_df.filter(col("total_votes") >= 20)
votes_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|RXAHWIC1584UQ |5          |20           |23         |N   |Y                |
|RSBELAIC899DO |5          |46           |51         |N   |Y                |
|R2HCC4CJ59D225|4          |43           |43         |N   |Y                |
|R39BVCCVPRV6F5|5          |10           |22         |N   |Y                |
|RL9VF9WXHEHKR |5          |35           |36         |N   |Y                |
|R1ZINLWUOLTZ46|1          |39           |45         |N   |N                |
|R1CQKM1K1CHOHS|1          |21           |22         |N   |N                |
|R1ZG19X1NOLCRF|5          |5            |21         |N   |Y                |
|R1YOX5Z4GF3ZSW|1          |1            |22         |N   |Y                |
|R2DHAM7J1KMWN |5          |23           |24         |N   |N    

In [7]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where 
# the number of helpful_votes divided by total_votes is equal to or greater than 50%.
top_votes_df = votes_df.filter((votes_df.helpful_votes / votes_df.total_votes) >= 0.5)
top_votes_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|RXAHWIC1584UQ |5          |20           |23         |N   |Y                |
|RSBELAIC899DO |5          |46           |51         |N   |Y                |
|R2HCC4CJ59D225|4          |43           |43         |N   |Y                |
|RL9VF9WXHEHKR |5          |35           |36         |N   |Y                |
|R1ZINLWUOLTZ46|1          |39           |45         |N   |N                |
|R1CQKM1K1CHOHS|1          |21           |22         |N   |N                |
|R2DHAM7J1KMWN |5          |23           |24         |N   |N                |
|R26EZ2INBETPU2|1          |103          |128        |N   |N                |
|R265ZMDO2ISN1O|5          |20           |21         |N   |Y                |
|R3NZ7JTOCPYE1S|4          |32           |33         |N   |Y    

In [8]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where 
# a review was written as part of the Vine program (paid), vine == 'Y'.
paid_df = top_votes_df.filter(top_votes_df.vine == "Y")
paid_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3QIQRG107AP19|5          |38           |41         |Y   |N                |
|R1ZNBHGCYE8VN1|5          |175          |181        |Y   |N                |
|R1BNZJ9IKGRBC2|4          |63           |70         |Y   |N                |
|R2IXTN4PBRHBO2|5          |19           |24         |Y   |N                |
|R32AOW3L09SYMQ|5          |29           |34         |Y   |N                |
|R34Z4VTLF7PC6X|4          |33           |36         |Y   |N                |
|R25403NE9JCRZZ|2          |36           |40         |Y   |N                |
|R2QT2G3YRGC0PS|5          |18           |20         |Y   |N                |
|RDUP15OURNUSA |5          |22           |23         |Y   |N                |
|R1EI4NZTG7G14F|5          |17           |23         |Y   |N    

In [9]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_df = top_votes_df.filter(top_votes_df.vine == "N")
unpaid_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|RXAHWIC1584UQ |5          |20           |23         |N   |Y                |
|RSBELAIC899DO |5          |46           |51         |N   |Y                |
|R2HCC4CJ59D225|4          |43           |43         |N   |Y                |
|RL9VF9WXHEHKR |5          |35           |36         |N   |Y                |
|R1ZINLWUOLTZ46|1          |39           |45         |N   |N                |
|R1CQKM1K1CHOHS|1          |21           |22         |N   |N                |
|R2DHAM7J1KMWN |5          |23           |24         |N   |N                |
|R26EZ2INBETPU2|1          |103          |128        |N   |N                |
|R265ZMDO2ISN1O|5          |20           |21         |N   |Y                |
|R3NZ7JTOCPYE1S|4          |32           |33         |N   |Y    

In [13]:
from pyspark.sql.functions import sum,avg,max,count
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews 
# for the two types of review (paid vs unpaid).
top_paid_df = paid_df.filter(paid_df.star_rating == 5)
top_paid_df.show(truncate=False)
paid_count = top_paid_df.agg(count("*").alias("total_paid_reviews"))
paid_count.show()

top_unpaid_df = unpaid_df.filter(unpaid_df.star_rating == 5)
top_unpaid_df.show(truncate=False)
unpaid_count = top_unpaid_df.agg(count("*").alias("total_unpaid_reviews"))
unpaid_count.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3QIQRG107AP19|5          |38           |41         |Y   |N                |
|R1ZNBHGCYE8VN1|5          |175          |181        |Y   |N                |
|R2IXTN4PBRHBO2|5          |19           |24         |Y   |N                |
|R32AOW3L09SYMQ|5          |29           |34         |Y   |N                |
|R2QT2G3YRGC0PS|5          |18           |20         |Y   |N                |
|RDUP15OURNUSA |5          |22           |23         |Y   |N                |
|R1EI4NZTG7G14F|5          |17           |23         |Y   |N                |
|R2G0YB0MC67NLG|5          |29           |38         |Y   |N                |
|R1QS5MKSKOGG0U|5          |15           |20         |Y   |N                |
|RSPSTXGY3OD8  |5          |29           |33         |Y   |N    